In [7]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold

In [4]:
# https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python

In [2]:
df_train = pd.read_csv("../data/raw/train.csv", na_values="NaN")

In [3]:
df_test = pd.read_csv("../data/raw/test.csv", na_values="NaN")

In [ ]:
df_predict = pd.DataFrame({"PassengerId": df_test["PassengerId"], "Survived": y_test})

In [ ]:
df_predict.to_csv("../data/processed/0.3_introduction_to_ensembling_stacking_in_python.csv", sep=",", index=False)